## This section is to gather the useful data within the big package that we have downloaded from the open-source website. In particular, the 3 different kinds of states: left, still and right hand state.

In [28]:
# Import the required Python libraries
import os
import re
from pathlib import Path
import pandas as pd

In [32]:
# Define the path to your raw_data directory
data_dir = Path('../data/raw data')  # Adjust this path as needed

# Verify the directory exists
if not data_dir.exists():
    raise FileNotFoundError(f"The specified data directory does not exist: {data_dir}")

# Define the specific runs to process, including both two-digit and three-digit run numbers
desired_runs = ['04', '08', '12']

# Compile a regular expression pattern to match filenames like S001R04.edf, S002R08.edf, S001R012.edf, etc.
# This pattern ensures that only R04, R08, R12 are matched
edf_pattern = re.compile(r'^S\d{3}R0?(04|08|12)\.edf$', re.IGNORECASE)

We’ll iterate through all files in the data_dir, filter out the .edf files that match the naming pattern, and collect their names.

In [33]:
# Initialize a list to store file information
file_info = []

# Counter for processed subject directories
processed_subjects = 0

# Iterate through each subject folder in raw_data
for subject_dir in data_dir.iterdir():
    if subject_dir.is_dir() and re.match(r'^S\d{3}$', subject_dir.name, re.IGNORECASE):
        processed_subjects += 1
        # Iterate through each file in the subject directory
        for file in subject_dir.iterdir():
            if file.is_file() and edf_pattern.match(file.name):
                # Derive the corresponding event file name
                base_name = file.stem  # e.g., S001R04
                event_file = f"{base_name}.edf.event"
                event_path = subject_dir / event_file
                
                # Check if the event file exists
                event_exists = event_path.exists()
                
                # Append the information to the list
                file_info.append({
                    'subject': subject_dir.name,
                    'edf_file': file.name,
                    'event_file': event_file,
                    'event_exists': event_exists,
                    'edf_path': str(file.resolve()),
                    'event_path': str(event_path.resolve()) if event_exists else None
                })

# Convert the list to a DataFrame for better visualization
df_files = pd.DataFrame(file_info)

# Display summary of processed subject directories
print(f"Total subject directories processed: {processed_subjects}")

Total subject directories processed: 109


In [34]:
# Total number of matched .edf files
total_edf = df_files.shape[0]

# Number of event files present
total_events = df_files['event_exists'].sum()

# Number of missing event files
missing_events = total_edf - total_events

print(f"Total matched .edf files: {total_edf}")
print(f"Number of corresponding event files found: {total_events}")
print(f"Number of missing event files: {missing_events}")

Total matched .edf files: 327
Number of corresponding event files found: 327
Number of missing event files: 0


In [35]:
# Filter the DataFrame for missing event files
df_missing_events = df_files[~df_files['event_exists']]

# Display the list of .edf files without corresponding event files
if not df_missing_events.empty:
    print("\nFiles missing corresponding event files:")
    display(df_missing_events[['subject', 'edf_file', 'event_file']])
else:
    print("\nAll matched .edf files have corresponding event files.")


All matched .edf files have corresponding event files.


In [36]:
# Display the entire DataFrame of matched files
print("List of All Matched .edf Files:")
display(df_files[['subject', 'edf_file', 'event_file', 'event_exists']])

List of All Matched .edf Files:


,subject,edf_file,event_file,event_exists
0,S038,S038R12.edf,S038R12.edf.event,True
1,S038,S038R04.edf,S038R04.edf.event,True
2,S038,S038R08.edf,S038R08.edf.event,True
3,S007,S007R08.edf,S007R08.edf.event,True
4,S007,S007R04.edf,S007R04.edf.event,True
...,...,...,...,...
322,S106,S106R04.edf,S106R04.edf.event,True
323,S106,S106R08.edf,S106R08.edf.event,True
324,S108,S108R04.edf,S108R04.edf.event,True
325,S108,S108R12.edf,S108R12.edf.event,True
